This ipython notebook has code inspired from this page: https://github.com/matterport/Mask_RCNN/blob/master/samples/demo.ipynb

# Importing Libraries and Setting Paths

In [1]:
import os

root = os.path.abspath('./')
# Directory of images to run detection on
coco_image_dir = os.path.join(root, 'dataset', "coco")
temporary_results = os.path.join(root, 'temp_results')

# Import usefull utilites
import numpy as np
import json
from addict import Dict
from tqdm import tqdm
import cv2
import pandas as pd

# Import deep learning modules
from mrcnn import utils
import mrcnn.model as modellib
import samples.coco.coco as coco

os.chdir(os.path.abspath("./git_repos/Mask_RCNN"))

# Local path to trained weights file
cwd = os.path.abspath('./')
weights_path = os.path.join(cwd, "mask_rcnn_coco.h5")

# Directory to save logs and trained model
model_dir = os.path.join(cwd, "logs")

Using TensorFlow backend.


# Loading the Dataset, the Model and the Weights!

In [2]:
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(coco_image_dir,"val","2017")
dataset.prepare()

loading annotations into memory...
Done (t=0.56s)
creating index...
index created!


In [3]:
# Create model object in inference mode.
config = coco.CocoConfig()
model = modellib.MaskRCNN(mode="inference", config = config, model_dir=model_dir )

# Load weights trained on MS-COCO
model.load_weights(weights_path, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


# Defining Functions

In [4]:
def initialize_files(json_path, csv_path):
    
    try:
        with open('{}/{}.json'.format(json_path, "Instance_Segmentation")) as json_file:
            predictions = json.load(json_file)
            image_id_df = pd.read_csv(csv_path, index_col=[0])
    except FileNotFoundError:
        predictions = []
        image_id_df = pd.DataFrame(data=0, index=["id"], columns=[0])
    
    return predictions, image_id_df

In [5]:
def instance_prediction(dataset, img_id, model):
    
    # Exctract image attributes
    image_info = Dict(dataset.image_info[img_id])
    image_path = image_info.path
    image_id = image_info.id
    image = cv2.imread(image_path)

    # Make an inference from the image
    preds = model.detect([image], verbose=0)
    pred = Dict(preds[0])

    # Make the inference compatible with COCO
    coco_instances = coco.build_coco_results(dataset,[image_id],pred.rois,pred.class_ids,pred.scores,
                                             pred.masks.astype(np.uint8))
    return coco_instances

In [6]:
def save_progress(json_path, csv_path, predictions, image_id_df, img_id):
    
    with open('{}/{}.json'.format(json_path, "Instance_Segmentation"), "w") as json_file:
        json.dump(predictions, json_file)
        image_id_df.iloc[0,0] = img_id
        image_id_df.to_csv(csv_path)
        print("Saved progress index: {}".format(img_id))

In [7]:
def instance_segmentation(model, dataset):

    # Initialize the JSON and CSV files
    csv_path = os.path.join(root, 'img_index.csv')
    json_path = temporary_results
    predictions, image_id_df = initialize_files(json_path, csv_path)

    # Construct instance predictions
    for img_id in tqdm(range(image_id_df.iloc[0,0], len(dataset.image_info))):
        image_segmentation = instance_prediction(dataset, img_id, model)
        predictions.extend(image_segmentation)
        save_progress(json_path, csv_path, predictions, image_id_df, img_id)
        
    #return predictions

# Run Inference

In [8]:
instance_segmentation(model, dataset)

  0%|          | 0/5000 [00:00<?, ?it/s]

Saved progress index: 0


  0%|          | 1/5000 [00:04<5:49:17,  4.19s/it]

Saved progress index: 1


  0%|          | 2/5000 [00:04<4:17:31,  3.09s/it]

...

100%|█████████▉| 4998/5000 [1:31:30<00:03,  1.58s/it]

Saved progress index: 4998


100%|█████████▉| 4999/5000 [1:31:31<00:01,  1.58s/it]

Saved progress index: 4999


100%|██████████| 5000/5000 [1:31:33<00:00,  1.62s/it]


In [ ]:
# instance_segmentation(model, dataset)

In [ ]:
# instance_segmentation(model, dataset)

In [ ]:
# instance_segmentation(model, dataset)